## Anchor Tabular Explainer for SKLearn Income Model

Let's start an income classification model that predicts whether a person will have high or low income based on demographic features.

In [1]:
!cat ./models/income.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income
spec:
  storageUri: "gs://seldon-models/scv2/examples/income/classifier"
  requirements:
  - sklearn


In [2]:
!seldon model load -f ./models/income.yaml

{}


In [3]:
!seldon model status income -w ModelAvailable

{}


We test the model - for this person we get a classification of 0 meaning "low income".

In [4]:
!seldon model infer income \
  '{"inputs": [{"name": "predict", "shape": [1, 12], "datatype": "FP32", "data": [[47,4,1,1,1,3,4,1,0,0,40,9]]}]}' 

{
	"model_name": "income_1",
	"model_version": "1",
	"id": "efd786f2-7d8f-46d7-a712-15f5f5c619b6",
	"parameters": {
		"content_type": null,
		"headers": null
	},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1
			],
			"datatype": "INT64",
			"parameters": null,
			"data": [
				0
			]
		}
	]
}


We have trained an Anchor Tabular explainer for this model. We start it. Note the parameters that define the explainer type and the "black box" model reference to our `income` model.

In [5]:
!cat ./models/income-explainer.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income-explainer
spec:
  storageUri: "gs://seldon-models/scv2/examples/income/explainer"
  explainer:
    type: anchor_tabular
    modelRef: income


In [6]:
!seldon model load -f ./models/income-explainer.yaml

{}


In [7]:
!seldon model status income-explainer -w ModelAvailable

{}


Let's now get a local explanation with the same input data.

In [9]:
!seldon model infer income-explainer \
  '{"inputs": [{"name": "predict", "shape": [1, 12], "datatype": "FP32", "data": [[47,4,1,1,1,3,4,1,0,0,40,9]]}]}'

{
	"model_name": "income-explainer_1",
	"model_version": "1",
	"id": "4f34ee1f-7710-4554-9dcd-221a2c0a125a",
	"parameters": {
		"content_type": null,
		"headers": null
	},
	"outputs": [
		{
			"name": "explanation",
			"shape": [
				1
			],
			"datatype": "BYTES",
			"parameters": {
				"content_type": "str",
				"headers": null
			},
			"data": [
				"{\"meta\": {\"name\": \"AnchorTabular\", \"type\": [\"blackbox\"], \"explanations\": [\"local\"], \"params\": {\"seed\": 1, \"disc_perc\": [25, 50, 75], \"threshold\": 0.95, \"delta\": 0.1, \"tau\": 0.15, \"batch_size\": 100, \"coverage_samples\": 10000, \"beam_size\": 1, \"stop_on_first\": false, \"max_anchor_size\": null, \"min_samples_start\": 100, \"n_covered_ex\": 10, \"binary_cache_size\": 10000, \"cache_margin\": 1000, \"verbose\": false, \"verbose_every\": 1, \"kwargs\": {}}, \"version\": \"0.7.0\"}, \"data\": {\"anchor\": [\"Marital Status = Never-Married\", \"Relationship = Own-child\", \"Capital Gain <= 0.00\"], \"precision\"

In [10]:
!seldon model unload income-explainer

{}


In [11]:
!seldon model unload income

{}
